### Rapid-prototype your own sampler

We will now implement a Geometric Langevin Algorithm 2nd order sampler.

This will show you how to use the `simulation` interface as a light-weight interface that lends itself well to rapid prototyping for general sampling of loss manifolds.

In [ ]:
import TATi.simulation as tati

The GLA2 sampler consists of the following steps given the current state $(q_, p_n)$: 

- B: $p^{n+\frac 1 2} = \nabla L(q^n )  \frac{\Delta t}{2}$
- A: $q^{n+\frac 1 2} = q^n + M^{−1} p^{n+\frac 1 2} \Delta t$
- B: $\tilde{p}^{n+1} = p^{n\frac 1 2} −\nabla L(q^{n+\frac 1 2}) \frac{\Delta t}{2}$
- O: $p^{n+1} = \alpha_{\Delta t} \tilde{p}^{n+1} + \sqrt{ \frac{1-\alpha^2_{\Delta t}}{\beta} M} G_n$, where $\alpha_{\Delta t} = \exp(−\gamma \Delta t)$


Now, we implement the actual GLA2 sampler in simple python code acting upon a given set of `parameters` and `momenta` using gradients that are obtained via a `gradients()` call.

In [ ]:
import math
import numpy as np

def gla2_update_step(nn, momenta, old_gradients, step_width, beta, gamma):
    """Implementation of GLA2 update step using TATi's simulation interface.
    
    Note:
        Parameters are contained inside nn. For momenta we use
        python variables as the evaluation of the loss does not
        depend on them.

    Args:
      nn: ref to tati simulation instance
      momenta: numpy array of parameters
      old_gradients: gradients evaluated at last step
      step_width: step width for sampling step
      beta: inverse temperature
      gamma: friction constant

    Returns:
      updated gradients and momenta

    """

    # 1. p_{n+\tfrac 1 2} = p_n - \tfrac {\lambda}{2} \nabla_x L(x_n)
    momenta -= .5*step_width * old_gradients

    # 2. x_{n+1} = x_n + \lambda p_{n+\tfrac 1 2}
    nn.parameters = nn.parameters + step_width * momenta

    # \nabla_x L(x_{n+1})
    gradients = nn.gradients()

    # 3. \widehat{p}_{n+1} = p_{n+\tfrac 1 2} - \tfrac {\lambda}{2} \nabla_x L(x_{n+1})
    momenta -= .5*step_width * gradients

    # 4. p_{n+1} = \alpha \widehat{p}_{n+1} + \sqrt{\frac{1-\alpha^2}{\beta}} \cdot \eta_n
    alpha = math.exp(-gamma*step_width)
    momenta = alpha * momenta + \
              math.sqrt((1.-math.pow(alpha,2.))/beta) * np.random.standard_normal(momenta.shape)

    return gradients, momenta


This function `gla2_update_step()` performs the integration steps for the GLA2.

Next, we need to instantiate the interface, handing parameter and defining the neural network.

In [ ]:
nn = tati(
    batch_data_files=["dataset-twoclusters.csv"],
    output_activation="linear",
    batch_size=5,
    seed=426,
)
print(nn.num_parameters())

Take note that we set a `batch_size` that is smaller than the dataset dimension. This is used to illustrate a point later on.

Before the iteration loop, we define some parameters needed by the GLA2 sampler.

In [ ]:
gamma = 10.
beta = 1e3

Moreover, we need temporary storage for the momentum and for the gradients.

In [ ]:
momenta = np.zeros((nn.num_parameters()))
old_gradients = np.zeros((nn.num_parameters()))

As the sampler's a source of random noise we will be using `numpy`'s random number generator. For reproducible runs we fix its seed.

In [ ]:
np.random.seed(426)

Finally, we set the neural network's `parameters` onto the minimum location found during training. Then, we proceedwith the actual sampling iteration that calls `gla2_update_step()` and prints the loss, parameters, and gradients.

In [ ]:
nn.parameters = np.array([0.14637233, 0.32722256, -0.045677684])
for i in range(10):
    old_gradients, momenta = gla2_update_step(
        nn, momenta, old_gradients, step_width=1e-1, beta=beta, gamma=gamma)
    print("Step #"+str(i)+": "+str(nn.loss())+" at " \
        +str(nn.parameters)+", gradients "+str(old_gradients))


### Summary

- how to implement a GLA2 sampler in python
- how to use it directly with `Simulation`
- how the next batch is triggered through a second evaluation of `gradients()`